# base_rbt

> Base functions and classes we use for our hacking on BT / RBT and related ideas

## Install

```sh
!pip install git+https://github.com/hamish-haggerty/base_rbt.git#egg='base_rbt'
```

## How to use

After installing, import like this:

In [ ]:
from base_rbt.base_model import *
from base_rbt.base_lf import *
from base_rbt.base_linear import *
from base_rbt.helper import *

We also need some other libraries:
